In [184]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from sklearn.model_selection import train_test_split
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()
import pickle
import gc
import lightgbm as lgb
warnings.filterwarnings('ignore')
%matplotlib inline

In [185]:
df = pd.read_csv("../input/home-credit-application-train/application_train.csv")
df

# Identify

In [186]:
df.info()

In [187]:
contract_val = df['NAME_CONTRACT_TYPE'].value_counts()
contract_df = pd.DataFrame({'labels': contract_val.index,
                   'values': contract_val.values
                  })
contract_df.iplot(kind='pie',labels='labels',values='values', title='Types of Loan', hole = 0.6)

In [188]:
df[df['AMT_INCOME_TOTAL'] < 2000000]['AMT_INCOME_TOTAL'].iplot(kind='histogram', bins=100,
   xTitle = 'Total Income', yTitle ='Count of applicants',
             title='Distribution of AMT_INCOME_TOTAL')

In [189]:
np.log(df['AMT_CREDIT']).iplot(kind='histogram', bins=100,
        xTitle = 'log(Credit Amount)',yTitle ='Count of applicants',
        title='Distribution of log(AMT_CREDIT)')

In [190]:
(df[df['AMT_INCOME_TOTAL'] > 1000000]['TARGET'].value_counts())/len(df[df['AMT_INCOME_TOTAL'] > 1000000])*100

In [191]:
import plotly.offline as py
import plotly.graph_objs as go
suite = df['NAME_TYPE_SUITE'].value_counts()

suite_val_y0 = []
suite_val_y1 = []
for val in suite.index:
    suite_val_y1.append(np.sum(df['TARGET'][df['NAME_TYPE_SUITE']==val] == 1))
    suite_val_y0.append(np.sum(df['TARGET'][df['NAME_TYPE_SUITE']==val] == 0))
data = [go.Bar(x = suite.index, y = ((suite_val_y1 / suite.sum()) * 100), name='Yes' ),
        go.Bar(x = suite.index, y = ((suite_val_y0 / suite.sum()) * 100), name='No' )]
layout = go.Layout(
    title = "Who accompanied client when applying for the  application in terms of loan is repayed or not in %",
    xaxis=dict(
        title='Name of type of the Suite',
       ),
    yaxis=dict(
        title='Count of applicants in %',
        )
)
fig = go.Figure(data = data, layout=layout) 
fig.layout.template = 'simple_white'
py.iplot(fig)


Pengguna yang tinggal bersama anaknya semua membayar pinjamannya

In [192]:
education_val = df['NAME_EDUCATION_TYPE'].value_counts()
education_val_y0 = []
education_val_y1 = []
for val in education_val.index:
    education_val_y1.append(np.sum(df['TARGET'][df['NAME_EDUCATION_TYPE']==val] == 1))
    education_val_y0.append(np.sum(df['TARGET'][df['NAME_EDUCATION_TYPE']==val] == 0))
data = [go.Bar(x = education_val.index, y = ((education_val_y1 / education_val.sum()) * 100), name='Yes' ),
        go.Bar(x = education_val.index, y = ((education_val_y0 / education_val.sum()) * 100), name='No' )]
layout = go.Layout(
    title = "Education sources of Applicants in terms of loan is repayed or not  in %",
    xaxis=dict(
        title='Education of Applicants',
       ),
    yaxis=dict(
        title='Count of applicants in %',
        )
)
fig = go.Figure(data = data, layout=layout) 
fig.layout.template = 'simple_white'
py.iplot(fig)

Pengguna yang memiliki gelar akademis/berpendidikan cenderung membayar pinjamannya, semakin tinggi tingkat pendidikannya, semakin bagus tingkat edukasi untuk membayar pinjaman

In [193]:
children_val = df['CNT_CHILDREN'].value_counts()
children_val_y0 = []
children_val_y1 = []
for val in children_val.index:
    children_val_y1.append(np.sum(df['TARGET'][df['CNT_CHILDREN']==val] == 1))
    children_val_y0.append(np.sum(df['TARGET'][df['CNT_CHILDREN']==val] == 0))
data = [go.Bar(x = children_val.index, y = ((children_val_y1 / children_val.sum()) * 100), name='Yes' ),
        go.Bar(x = children_val.index, y = ((children_val_y0 / children_val.sum()) * 100), name='No' )]
layout = go.Layout(
    title = "Children count of Applicants in terms of loan is repayed or not  in %",
    xaxis=dict(
        title='Children count of Applicants',
       ),
    yaxis=dict(
        title='Count of applicants in %',
        )
)
fig = go.Figure(data = data, layout=layout) 
fig.layout.template = 'simple_white'
py.iplot(fig)

In [194]:
car_val = df['FLAG_OWN_CAR'].value_counts()
car_val_y0 = []
car_val_y1 = []
for val in car_val.index:
    car_val_y1.append(np.sum(df['TARGET'][df['FLAG_OWN_CAR']==val] == 1))
    car_val_y0.append(np.sum(df['TARGET'][df['FLAG_OWN_CAR']==val] == 0))
data = [go.Bar(x = car_val.index, y = ((car_val_y1 / car_val.sum()) * 100), name='Yes' ),
        go.Bar(x = car_val.index, y = ((car_val_y0 / car_val.sum()) * 100), name='No' )]
layout = go.Layout(
    title = "Flag owning car of Applicants in terms of loan is repayed or not  in %",
    xaxis=dict(
        title='Flag owning car of Applicants',
       ),
    yaxis=dict(
        title='Count of applicants in %',
        )
)
print(car_val_y0,car_val_y1)
fig = go.Figure(data = data, layout=layout) 
fig.layout.template = 'simple_white'
py.iplot(fig)

tidak ada perbedaan yg signifikan antara mempunyai mobil atau tidak

# Preprocessing and Preparation

In [195]:
count = df.isnull().sum().sort_values(ascending=False)
percentage = ((df.isnull().sum()/len(df)*100)).sort_values(ascending=False)
missing_application = pd.concat([count, percentage], axis=1, keys=['Count','Percentage'])
print('Count and percentage of missing values for top 20 columns:')
missing_application.head(20)

In [196]:
def missing_values(df, percentage):

    columns = df.columns
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': columns,
                                 'percent_missing': percent_missing})

    missing_drop = list(missing_value_df[missing_value_df.percent_missing>percentage].column_name)
    df = df.drop(missing_drop, axis=1)
    return df

df = missing_values(df,60)
df

In [197]:
df = df.replace(to_replace = np.nan, value = 0)
df.isnull().sum()

In [198]:
columns_without_id = [col for col in df.columns if col!='SK_ID_CURR']
#Checking for duplicates in the data.
df[df.duplicated(subset = columns_without_id, keep=False)]
print('The no of duplicates in the data:',df[df.duplicated(subset = columns_without_id, keep=False)]
      .shape[0])

## Feature Selection

In [199]:
# Find correlation with the target and sort
correlations = df.corr()['TARGET'].sort_values()

# Display correlations
print('Most Postive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

In [200]:
first = df[['TARGET', 'EXT_SOURCE_3', 'EXT_SOURCE_1', 'EXT_SOURCE_2','DAYS_BIRTH','DAYS_EMPLOYED','REG_CITY_NOT_WORK_CITY','DAYS_ID_PUBLISH','DAYS_LAST_PHONE_CHANGE','REGION_RATING_CLIENT','REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY','CODE_GENDER', 'NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE']]
first

In [201]:
#Map the atribute into ordinal numbers from 0-4
scale_mapper = {'Lower secondary' : 0, 'Secondary / secondary special' : 1, 
'Incomplete higher' : 2, 'Higher education' : 3, 'Academic degree' : 4}

first['NAME_EDUCATION_TYPE'] = first['NAME_EDUCATION_TYPE'].replace(scale_mapper)
first['NAME_EDUCATION_TYPE'].unique()

In [202]:
#Create dummies/one-hot encoding on every nominal attributes in train dataframe
CODE_GENDER_dummies = pd.get_dummies(first['CODE_GENDER'],prefix='CODE_GENDER')
NAME_INCOME_TYPE_dummies = pd.get_dummies(first['NAME_INCOME_TYPE'],prefix='NAME_INCOME_TYPE')

#Concate the dummies into train dataset
first = pd.concat([first, CODE_GENDER_dummies, NAME_INCOME_TYPE_dummies], axis=1)

#Drop categorical attribute (before transformation)
first = first.drop(['CODE_GENDER','NAME_INCOME_TYPE'], axis=1)

In [203]:
first.head()

# Modelling

In [204]:
first.info()

In [205]:
y = first.iloc[:, 0].values
x = first.iloc[:, 1:23].values

In [206]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_smote, y_smote = smote.fit_resample(x, y)

In [207]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_smote = sc.fit_transform(x_smote)

In [208]:
x_train, x_test, y_train, y_test = train_test_split(x_smote, y_smote, test_size = 0.3, random_state=10)

## First Try ( Logistic Regression )

In [209]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
prediction = logreg.predict(x_test)

In [210]:
akurasi = accuracy_score(y_test, prediction)
akurasi

In [211]:
auc = roc_auc_score(y_test, prediction)
print(auc)

In [212]:
klasifikasi = classification_report(y_test, prediction)
print(klasifikasi)

## Second Try ( Random Forest Classifier )

In [213]:
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)

In [214]:
score = accuracy_score(y_test, y_pred)
print(score)

In [215]:
auc = roc_auc_score(y_test, y_pred)
print(auc)

In [216]:
klasifikasi = classification_report(y_test, y_pred)
print(klasifikasi)